In [301]:
from pathlib import Path
from decode_fish.funcs.utils import *
import glob

def make_sweep_str(pfile, pars, group, resume=False, init=None, gpu='gpu_rtx', runfile='train.py', n=1):
    
    base_str = f'bsub -gpu "num=1" -n {n} -q {gpu} -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/{runfile}'   
    
    if pfile is not None:
        base_str = base_str + f' +experiment={pfile}'
    name = ''
    
    for k in pars:
        if k!= 'run_name':
            base_str += ' ' + k + f'={pars[k]}'
            name += k.split('.')[-1]+f':{pars[k]}x'
            name = name.replace('[', '')
            name = name.replace(']', '')
            name = name.replace(',', 'x')
    name = name[:-1]
    
    if 'run_name' in pars:
        name=pars['run_name']
    
    if resume:
        base_str += ' training.resume=True '
    if init is not None:
        base_str += f' data_path.model_init={init} '
    
    return base_str + f' run_name={name} output.group={group}' + '\n'
    

def make_named_str(pfile, name, group):
    
    base_str = base_str + f' +experiment={pfile}'
    
    return base_str + f' run_name={name} output.group={group}' + '\n'

In [179]:
# cfg.training.mic.par_grads.channel_facs = False
# cfg.training.mic.par_grads.channel_shifts = False
# cfg.training.mic.par_grads.theta_par = False
# cfg.training.mic.par_grads.psf_vol = False

In [186]:
group = 'sweep_mop_20'
pfile = 'MERFISH_mop_20'

variable_col = param_iter()
variable_col.add('data_path.image_path', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_images150.tif', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_images151.tif', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_images47.tif')
par_prod = variable_col.param_product()

In [68]:
group = 'sim_int_2'
pfile = 'sim_int_2'

variable_col = param_iter()
variable_col.add('training.mic.par_grads.channel_facs', False)
# variable_col.add('training.mic.par_grads.channel_shifts', False)
# variable_col.add('training.mic.par_grads.psf_vol', False)
par_prod = variable_col.param_product()

In [292]:
group = 'sweep_mop_32'
pfile = 'MERFISH_mop_32'

variable_col = param_iter()
variable_col.add('training.mic.par_grads.z_facs', False)
variable_col.add('genm.microscope.norm', 'none')
# variable_col.add('data_path.image_path', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled4_images47.tif', 
#                                         '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled4_images150.tif',
#                                         '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled4_images151.tif')
par_prod = variable_col.param_product()

In [293]:
# Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(pfile, pars, group, resume=False, gpu='gpu_rtx') #, 
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_32 training.mic.par_grads.z_facs=False genm.microscope.norm=none run_name=z_facs:Falsexnorm:none output.group=sweep_mop_32



In [236]:
bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_32 training.target_mean=True data_path.image_path=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled4_images47.tif run_name=target_mean:Trueximage_path:scaled4_images47.tif output.group=sweep_mop_32

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_32 training.target_mean=True data_path.image_path=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled4_images150.tif run_name=target_mean:Trueximage_path:scaled4_images150.tif output.group=sweep_mop_32

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_32 training.target_mean=True data_path.image_path=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//datasets/CodFish/MERFISH/MOp/mouse1_sample1_raw/aligned_scaled4_images151.tif run_name=target_mean:Trueximage_path:scaled4_images151.tif output.group=sweep_mop_32

SyntaxError: invalid syntax (3663348337.py, line 1)

In [290]:
group = 'MERFISH_sweep_gen_9'
pfile = 'MERFISH_sweep_gen_9'

variable_col = param_iter()
variable_col.add('training.mic.par_grads.z_facs', False)
variable_col.add('genm.microscope.norm', 'none')
# variable_col.add('training.mic.ch_facs_as_theta', True)
# variable_col.add('data_path.model_init', '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/MERFISH_sweep_gen_7/seed:0xphasing:0.1//checkpoint/')

par_prod = variable_col.param_zip()

In [291]:
# Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(pfile, pars, group, resume=False, gpu='gpu_rtx') #, 
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_sweep_gen_9 training.mic.par_grads.z_facs=False genm.microscope.norm=none run_name=z_facs:Falsexnorm:none output.group=MERFISH_sweep_gen_9



In [264]:
bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_sweep_gen_7 seed=0 genm.phasing=0.1 data_path.model_init=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/MERFISH_sweep_gen_7/seed:0xphasing:0.1//checkpoint/ run_name=seed:0xphasing:0.1 training.resume=True output.group=MERFISH_sweep_gen_7

SyntaxError: invalid syntax (3707715445.py, line 1)

In [386]:
group = 'mettrain_sweep_gen_9'

variable_col = param_iter()
variable_col.add('ignore', 0, 1, 2, 3, 4, 5, 102345)
variable_col.add('rate_fac', 5.)
par_prod = variable_col.param_product()

In [387]:
with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(None, pars, group, resume=False, gpu='gpu_rtx', runfile='metricnettrain.py') #, 
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/metricnettrain.py ignore=0 rate_fac=5.0 run_name=ignore:0xrate_fac:5.0 output.group=mettrain_sweep_gen_9

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/metricnettrain.py ignore=1 rate_fac=5.0 run_name=ignore:1xrate_fac:5.0 output.group=mettrain_sweep_gen_9

bsub -gpu "num=1" -n 1 -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/metricnettrain.py ignore=2 rate_fac=5.0 run_name=ignore:2xrate_fac:5.0 output.group=mettrain_sweep_gen_9

b